In [13]:
#Fetch our league data
from espn_api.football import League

# Initialize your league
league_id = 26347
espn_s2 = 'AECJTHUB5QQ41P4C5vinQpk7fGVA6h%2BnbM7tsN7mhlpWupwMWzVIKnKFd219nyX3Ss37wALT0z0fYoIOd9zieRZOE6I3nG%2BSSEUksFfA43gw8Hv3ywuj9PXh1fTxJlA9O%2FPfzY9GgfQH1OwPqQsmvWx0Zt7YOZKaBvy1ORbTZfgMfOZCVkqNYWMpBZzHCzAun99t%2FS3i24onjEXOch2vI9E%2Ff4y5%2BRBiE%2BaPaOlfnMTy1d3DbG1E%2FYqnZzNWbT3Yk3%2FFq7cLHbHTL1HF4Ouvgf6N'
swid = '{C3FE8278-A2E3-4D18-86D2-0154124A1F16}'
year = 2023  # Replace with the specific year you want

# Initialize the league for the specific year
league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid, debug=False)


In [14]:
# Pull all matchups
matchup_scores = []

# Iterate through each week
for week in range(1, league.current_week + 1):
    # Get the box scores for the current week
    box_scores = league.box_scores(week)

    # Iterate through each game in the box scores
    for game_id, game in enumerate(box_scores, start=1):
        home_team = game.home_team
        away_team = game.away_team

        # Retrieve the scores for each team in the game
        home_score = game.home_score
        away_score = game.away_score

        # Additional information
        season = year
        home_projected = game.home_projected
        away_projected = game.away_projected 

        matchup_info = {
            "game_id": game_id,
            "season": season,
            "week": week,
            "home_team": home_team.team_name,
            "home_team_id": home_team.team_id,
            "home_score": home_score,
            "away_team": away_team.team_name,
            "away_team_id": away_team.team_id,
            "away_score": away_score,
            "home_projected": home_projected,
            "away_projected": away_projected,
        }

        matchup_scores.append(matchup_info)

# Now, matchup_scores contains information about scores, season, projections, and other details for each matchup in every week


In [15]:
#Data cleaning and column creation
import pandas as pd
matchups_df = pd.DataFrame(matchup_scores)
# Add the "home_team_win" label to the DataFrame based on the condition
matchups_df['home_team_win'] = (matchups_df['home_score'] > matchups_df['away_score']).astype(int)
matchups_df['winner_id'] = matchups_df.apply(lambda row: row['home_team_id'] if row['home_team_win'] == 1 else row['away_team_id'], axis=1)
matchups_df = matchups_df[matchups_df['home_score'] != 0]
matchups_df['matchup_id'] =  matchups_df['season'].astype(str) + matchups_df['week'].astype(str) + matchups_df['game_id'].astype(str)
matchups_df.head()

,game_id,season,week,home_team,home_team_id,home_score,away_team,away_team_id,away_score,home_projected,away_projected,home_team_win,winner_id,matchup_id
0,1,2023,1,Team SOOO GOOD,9,85.0,Team Gibson,4,116.0,97.55,95.77,0,4,202311
1,2,2023,1,Gastropod Gunmen,8,80.0,The U23 Bobby Football Team,6,59.0,89.23,91.24,1,8,202312
2,3,2023,1,Monty Men,1,91.0,Jack's Krevice,3,89.0,87.58,96.32,1,1,202313
3,4,2023,1,Team LoSardo,2,75.0,DA LOVE DOCTER,5,71.0,95.08,87.05,1,2,202314
4,5,2023,1,Team Gibsin,10,52.0,Demarco Murray,7,95.0,98.54,94.69,0,7,202315


In [16]:
#Create a historical standings table

# Create an empty DataFrame to store the standings
standings_df = pd.DataFrame()

# Get a list of all unique team IDs
team_ids = matchups_df['home_team_id'].unique()

# Determine the maximum number of weeks in the dataset
max_week = matchups_df['week'].max()

# Iterate through each team
for team_id in team_ids:
    # Create a DataFrame for the current team with all weeks' statistics
    team_df = pd.DataFrame({
        'team_id': [team_id] * max_week,
        'prior_to_week': list(range(1, max_week + 1)),
        'wins': 0,
        'losses': 0,
        'points_for': 0,
        'points_against': 0,
        'win_percentage': 0,
        'points_per_game': 0,
        'points_against_per_game': 0
    })

    # Iterate through each week
    for week in range(1, max_week + 1):
        # Filter the DataFrame to get data prior to the current week
        prior_to_week_df = matchups_df[matchups_df['week'] < week]

        # Filter the DataFrame to get matches involving the current team
        team_matches = prior_to_week_df[(prior_to_week_df['home_team_id'] == team_id) | (prior_to_week_df['away_team_id'] == team_id)]

        # Calculate team statistics
        team_wins = sum(team_matches['home_team_id'] == team_id)
        team_losses = sum(team_matches['away_team_id'] == team_id)
        team_points_for = sum(team_matches.loc[team_matches['home_team_id'] == team_id, 'home_score']) + sum(team_matches.loc[team_matches['away_team_id'] == team_id, 'away_score'])
        team_points_against = sum(team_matches.loc[team_matches['home_team_id'] == team_id, 'away_score']) + sum(team_matches.loc[team_matches['away_team_id'] == team_id, 'home_score'])
        total_games = team_wins + team_losses
        win_percentage = team_wins / total_games if total_games > 0 else 0
        points_per_game = team_points_for / total_games if total_games > 0 else 0
        points_against_per_game = team_points_against / total_games if total_games > 0 else 0

        # Update the current week's statistics in the team's DataFrame
        team_df.loc[week - 1, 'wins'] = team_wins
        team_df.loc[week - 1, 'losses'] = team_losses
        team_df.loc[week - 1, 'points_for'] = team_points_for
        team_df.loc[week - 1, 'points_against'] = team_points_against
        team_df.loc[week - 1, 'win_percentage'] = win_percentage
        team_df.loc[week - 1, 'points_per_game'] = points_per_game
        team_df.loc[week - 1, 'points_against_per_game'] = points_against_per_game

    # Append the team's DataFrame to the standings DataFrame
    standings_df = pd.concat([standings_df, team_df], ignore_index=True)

In [17]:
# Merge historical standings into matchup data to get team stats prior to each matchup

# Merge 'standings_df' into 'df' for home team's statistics
matchups_df = pd.merge(matchups_df, standings_df, how='left', left_on=['home_team_id', 'week'], right_on=['team_id', 'prior_to_week'])

# Rename the columns for home team's statistics
matchups_df = matchups_df.rename(columns={
    'win_percentage': 'home_team_win_pct',
    'points_per_game': 'home_team_ppg'
})

# Drop the redundant columns from the merge
matchups_df = matchups_df.drop(['team_id', 'prior_to_week', 'wins', 'losses', 'points_for', 'points_against', 'points_against_per_game'], axis=1)

# Merge 'standings_df' into 'df' for away team's statistics
matchups_df = pd.merge(matchups_df, standings_df, how='left', left_on=['away_team_id', 'week'], right_on=['team_id', 'prior_to_week'])

# Rename the columns for away team's statistics
matchups_df = matchups_df.rename(columns={
    'win_percentage': 'away_team_win_pct',
    'points_per_game': 'away_team_ppg'
})

# Drop the redundant columns from the merge
matchups_df = matchups_df.drop(['team_id', 'prior_to_week', 'wins', 'losses', 'points_for', 'points_against', 'points_against_per_game'], axis=1)

# Print the updated 'df' DataFrame
matchups_df.head(30)

,game_id,season,week,home_team,home_team_id,home_score,away_team,away_team_id,away_score,home_projected,away_projected,home_team_win,winner_id,matchup_id,home_team_win_pct,home_team_ppg,away_team_win_pct,away_team_ppg
0,1,2023,1,Team SOOO GOOD,9,85.0,Team Gibson,4,116.0,97.55,95.77,0,4,202311,0.000000,0.000000,0.000000,0.000000
1,2,2023,1,Gastropod Gunmen,8,80.0,The U23 Bobby Football Team,6,59.0,89.23,91.24,1,8,202312,0.000000,0.000000,0.000000,0.000000
2,3,2023,1,Monty Men,1,91.0,Jack's Krevice,3,89.0,87.58,96.32,1,1,202313,0.000000,0.000000,0.000000,0.000000
3,4,2023,1,Team LoSardo,2,75.0,DA LOVE DOCTER,5,71.0,95.08,87.05,1,2,202314,0.000000,0.000000,0.000000,0.000000
4,5,2023,1,Team Gibsin,10,52.0,Demarco Murray,7,95.0,98.54,94.69,0,7,202315,0.000000,0.000000,0.000000,0.000000
5,1,2023,2,Team Gibson,4,147.0,DA LOVE DOCTER,5,74.0,100.70,88.77,1,4,202321,0.000000,116.000000,0.000000,71.000000
6,2,2023,2,Gastropod Gunmen,8,61.0,Team SOOO GOOD,9,95.0,85.64,99.45,0,9,202322,1.000000,80.000000,1.000000,85.000000
7,3,2023,2,Monty Men,1,82.0,The U23 Bobby Football Team,6,118.0,95.10,92.78,0,6,202323,1.000000,91.000000,0.000000,59.000000
8,4,2023,2,Team LoSardo,2,85.0,Demarco Murray,7,110.0,97.32,97.57,0,7,202324,1.000000,75.000000,0.000000,95.000000
9,5,2023,2,Team Gibsin,10,103.0,Jack's Krevice,3,89.0,91.53,93.49,1,10,202325,1.000000,52.000000,0.000000,89.000000
